## 回帰分析
出力変数には因果効果が表れる変数を使用し、  
入力変数にはバックドアパスを閉じるように因果ダイアグラムをd分離した後に残っている変数のみを利用する。

ここでテレビCMの架空事例を考える。  
「iさん」の年齢を$x_1^i$、性別を$x_2^i$として表す。  
「iさん」がCMを見た場合$Z^i=1$となり、$Z^i_1$と表す。CMを見ていない場合は$Z^i_0$となる。  
「iさん」の商品購入量を$Y^i$と記載し、$Z^i=1$の場合は$Y^i_1$、$Z^i=0$の場合は$Y^i_0$と記載される。  
  
年齢$x_1^i$と性別$x_2^i$は交絡因子であるので、d分離するためには$x_1^i$と$x_2^i$を考慮する必要がある。  
$$
Y^i_1=f_1(Z^i,x_1^i,x_2^i)\\
Y^i_0=f_0(Z^i,x_1^i,x_2^i)
$$
線形回帰モデルで書き直すと、
$$
Y^i_1=b_{z1}Z^i_1+b_1x_1^i+b_2x_2^i+b_0=b_z×1+b_1x_1^i+b_2x_2^i+b_0\\
Y^i_1=b_{z0}Z^i_0+b_1x_1^i+b_2x_2^i+b_0=b_z×0+b_1x_1^i+b_2x_2^i+b_0
$$
$Z$は0か1の2値なので、$b_{z1}=b_{z0}=b_z$と書き直しても問題はない。  
最終的に回帰モデルは
$$
Y^i=b_zZ^i+b_1x_1^i+b_2x_2^i+b_0
$$

In [5]:
import numpy as np
from numpy.random import randint, randn

num_data = 200

x_1 = randint(15, 76, num_data)
x_2 = randint(0, 2, num_data)

In [21]:
e_z = randn(num_data)

z_base = x_1 + (x_2)+10 - 40 + 5*e_z

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

z_prob = sigmoid(0.1*z_base)

Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)

In [22]:
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [23]:
import pandas as pd

df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y})

df.head()

,年齢,性別,CMを見た,購入量
0,42,1,0.0,77.603291
1,19,1,0.0,94.670630
2,25,1,1.0,115.727395
3,18,1,0.0,100.441828
4,41,1,1.0,78.515094


In [24]:
print(df[df['CMを見た']==1.0].mean())
print('--------------------')
print(df[df['CMを見た']==0.0].mean())

年齢       50.623188
性別        0.528986
CMを見た     1.000000
購入量      54.338187
dtype: float64
--------------------
年齢       30.032258
性別        0.548387
CMを見た     0.000000
購入量      67.340469
dtype: float64


In [25]:
from sklearn.linear_model import LinearRegression

X = df[["年齢", "性別", "CMを見た"]]
y = df[["購入量"]]

reg = LinearRegression().fit(X, y)

print('係数:', reg.coef_)

係数: [[-1.0644072  31.20355516  9.52025179]]


### 逆確率重み付け法(IPTW)
調整化公式
$$
P(Y=y|do(Z=z))=\sum_xP(Y=y|Z=z,X=x)P(X=x)
$$
ここで、右辺の分母分子に$P(Z=z|X=x)$を掛けると、
$$
\sum_xP(Y=y|Z=z,X=x)P(X=x)=\sum_x\frac{P(Y=y,X=x|Z=z)P(Z=z|X=x)P(X=x)}{P(Z=z|X=x)}
$$
さらに、同時確率と条件付き確率の変換を行い、
$$
\sum_xP(Y=y|Z=z,X=x)P(X=x)=\sum_x\frac{P(Y=y,Z=z,X=x)}{P(Z=z|X=x)}
$$
以上より、調整化公式は
$$
P(Y=y|do(Z=z))=\sum_x\frac{P(x,y,z)}{P(z|x)}
$$
ここで、$P(z|x)$を**傾向スコア**と呼ぶ。

In [27]:
from sklearn.linear_model import LogisticRegression

X = df[["年齢", "性別"]]
Z = df["CMを見た"]

reg = LogisticRegression().fit(X,Z)

print("係数beta :", reg.coef_)
print("係数alpha :", reg.intercept_)

係数beta : [[ 0.09622365 -0.20832621]]
係数alpha : [-2.87987577]


In [28]:
Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])
print("--------")
print(Z[0:5])

[[0.27824346 0.72175654]
 [0.77901795 0.22098205]
 [0.66432112 0.33567888]
 [0.79513753 0.20486247]
 [0.29797326 0.70202674]]
--------
0    0.0
1    0.0
2    1.0
3    0.0
4    1.0
Name: CMを見た, dtype: float64


平均処置効果は、
$$
ATE = E(Y_1)-E(Y_0)=E(Y_1|do(Z=1))-E(Y_0|do(Z=0))
$$
であり、調整化公式で書き換えると、
$$
ATE = \sum_xP(Y|Z=1,X=x)P(X=x)-\sum_xP(Y|Z=0,X=x)P(X=x)
$$
さらにさきほどの式変形より
$$
ATE = \sum_x\frac{P(Y,Z=1,X=x)}{P(Z=1|X=x)}-\sum_x\frac{P(Y,Z=0,X=x)}{P(Z=0|X=x)}
$$
Yが連続値の場合
$$
ATE = \frac{1}{N}\sum_i^N\frac{y_i}{P(Z=1|X=x_i)}Z_i-\frac{1}{N}\sum_i^N\frac{y_i}{P(Z=0|X=x_i)}(1-Z_i)
$$

In [29]:
ATE_i = Y/Z_pre[:,1]*Z-Y/Z_pre[:,0]*(1-Z)
ATE = 1/len(Y)*ATE_i.sum()
print("推定したATE", ATE)

推定したATE 9.482524478717783


## Doubly Robust法(DR法)

平均処置TEの前半の項を
$$
\frac{y_i}{P(Z=1|X=x_i)}Z_i+\bigr(1-\frac{Z_i}{P(Z=0|X=x_i)}\bigl)\hat{Y}^i_1
$$
とする。反実仮想を計算に加えることができる。  
同様に後半の項は、
$$
\frac{y_i}{P(Z=0|X=x_i)}(1-Z_i)+\bigr(1-\frac{(1-Z_i)}{P(Z=0|X=x_i)}\bigl)\hat{Y}^i_0
$$
ここで、$\hat{Y}$は回帰分析から求めた推測値である。  
このようにIPTW法での推定と回帰分析での推定を組み合わせて因果の効果を推定する方法を**Doubly Robust Estimation**と呼ぶ。

In [33]:
from sklearn.linear_model import LinearRegression

X = df[["年齢", "性別", "CMを見た"]]
y = df["購入量"]

reg2 = LinearRegression().fit(X,y)

X_0 = X.copy()
X_0["CMを見た"] = 0
Y_0 = reg2.predict(X_0)

X_1 = X.copy()
X_1["CMを見た"] = 1
Y_1 = reg2.predict(X_1)

In [35]:
from sklearn.linear_model import LogisticRegression

X = df[["年齢", "性別"]]
Z = df["CMを見た"]

reg = LogisticRegression().fit(X,Z)

Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])

[[0.27824346 0.72175654]
 [0.77901795 0.22098205]
 [0.66432112 0.33567888]
 [0.79513753 0.20486247]
 [0.29797326 0.70202674]]


In [37]:
ATE_1_i = Y/Z_pre[:,1]*Z + (1-Z/Z_pre[:,1])*Y_1
ATE_0_i = Y/Z_pre[:,0]*(1-Z) + (1-(1-Z)/Z_pre[:,0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()
print("推定したATE", ATE)

推定したATE 9.364474321296356
